## Required Libraries

In [2]:
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import to_networkx
import networkx as nx
import collections
import random
from networkx.algorithms import isomorphism
import itertools
from collections import Counter
import math
import numpy as np
from networkx.algorithms import distance_regular
import random

In [3]:
random.seed(42)
torch.manual_seed(42)

## Experiment A

In [4]:
dataset = TUDataset(root='/tmp/AIDS', name='AIDS', use_node_attr=True)
print(f"Loaded {len(dataset)} graphs.")

Loaded 2000 graphs.


In [5]:
def weisfeiler_lehman_hash(graph):
    colors = {node: data.get('label', 0) for node, data in graph.nodes(data=True)}
    for _ in range(len(graph.nodes())):
        new_colors = {}
        for node in graph.nodes():
            neighbor_colors = sorted([colors[nbr] for nbr in graph.neighbors(node)])
            signature = (colors[node], tuple(neighbor_colors))
            new_colors[node] = hash(signature)
        if new_colors == colors:
            break
        colors = new_colors
    return str(sorted(colors.values()))


In [6]:
def find_isomorphic_groups(dataset):
    hashes = collections.defaultdict(list)
    for i, data in enumerate(dataset):
        g = to_networkx(data, node_attrs=['x'])
        nx.set_node_attributes(g, {j: int(x[0]) for j, x in enumerate(data.x)}, 'label')
        if len(g) > 0:
            h = weisfeiler_lehman_hash(g)
            hashes[h].append(i)
    return {h: idxs for h, idxs in hashes.items() if len(idxs) > 1}


In [7]:
isomorphic_groups = find_isomorphic_groups(dataset)
print(f"No of isomorphic Graphs: {len(isomorphic_groups)}")

No of isomorphic Graphs: 84


In [8]:
largest_group = max(isomorphic_groups.values(), key=len)
print(f"Largest group has {len(largest_group)} i.e. graphs {largest_group[:10]}")

Largest group has 8 i.e. graphs [709, 1270, 1454, 1585, 1607, 1809, 1964, 1977]


In [9]:
def perturb_graph(graph):
    g_copy =graph.copy()
    possible_ops =[]
    n_nodes =len(g_copy.nodes())
    max_edges =n_nodes*(n_nodes-1)//2
    if len(g_copy.edges()) > 0:
        possible_ops.append('remove')
    if len(g_copy.edges()) < max_edges:
        possible_ops.append('add')
    if not possible_ops:
        return g_copy, "null_graph"
    op =random.choice(possible_ops)
    if op =='add':
        edge_to_add =random.choice(list(nx.non_edges(g_copy)))
        g_copy.add_edge(*edge_to_add)
        return g_copy,f"added_{edge_to_add}"
    else:
        edge_to_remove =random.choice(list(g_copy.edges()))
        g_copy.remove_edge(*edge_to_remove)
        return g_copy,f"removed_{edge_to_remove}"


In [10]:
original_graphs = []
for idx in largest_group:
    data =dataset[idx]
    g = to_networkx(data, node_attrs=['x'])
    nx.set_node_attributes(g, {j: int(x[0]) for j, x in enumerate(data.x)},'label')
    original_graphs.append(g)

In [11]:
perturbed_graphs = []
metadata = []
for i, g in enumerate(original_graphs):
    for j in range(10):
        pg, op = perturb_graph(g)
        perturbed_graphs.append(pg)
        metadata.append((i,j,op))
print(f"Total No of pertubed Graphs : {len(perturbed_graphs)}")

Total No of pertubed Graphs : 80


In [12]:
original_hashes = [weisfeiler_lehman_hash(g) for g in original_graphs]
original_hash = original_hashes[0]

In [13]:
perturbed_hashes =[weisfeiler_lehman_hash(g) for g in perturbed_graphs]

In [14]:
maintained = sum(1 for h in perturbed_hashes if h == original_hash)
broken = len(perturbed_hashes) - maintained

In [15]:
print(f"Original group size: {len(original_graphs)}")
print(f"Perturbed graphs: {len(perturbed_graphs)}")
print(f"Maintained isomorphism: {maintained}")
print(f"Broken isomorphism: {broken}")
print(f"Maintenance rate: {maintained/len(perturbed_graphs):.2%}")

Original group size: 8
Perturbed graphs: 80
Maintained isomorphism: 0
Broken isomorphism: 80
Maintenance rate: 0.00%


**All of the pertubed graphs were initially isomorphic(before pertubation) became non isomorphic after a small modification.This shows that WL is highly discriminative i.e. even a small perturbation is enough to alter its outcome**

## Experiment B

In [16]:
isomorphic_groups = find_isomorphic_groups(dataset)
sorted_groups = sorted(isomorphic_groups.values(), key=len, reverse=True)

In [17]:
print(f"Total WL-isomorphic groups: {len(sorted_groups)}")
print(f"Largest group size: {len(sorted_groups[0])} i.e. {sorted_groups[0][:10]}")
print(f"Second largest group size: {len(sorted_groups[1])} i.e. {sorted_groups[1][:10]}")

Total WL-isomorphic groups: 84
Largest group size: 8 i.e. [709, 1270, 1454, 1585, 1607, 1809, 1964, 1977]
Second largest group size: 3 i.e. [189, 1004, 1711]


In [18]:
def degree_sequence(g):
    return sorted([d for _, d in g.degree()])

In [19]:
def two_hop_neighbor_degrees(g):
    per_node = []
    for v in g.nodes():
        two_hop = set()
        for nbr in g.neighbors(v):
            for nbr2 in g.neighbors(nbr):
                if nbr2 != v:
                    two_hop.add(nbr2)
        degs = tuple(sorted([g.degree(u) for u in two_hop]))
        per_node.append(degs)
    return sorted(per_node)

In [20]:
def triangle_count(g):
    return sum(nx.triangles(g).values()) // 3

def four_cycle_count(g):
    nodes = list(g.nodes())
    count = 0
    for quad in itertools.combinations(nodes, 4):
        sub = g.subgraph(quad)
        if sub.number_of_edges() == 4:
            degs = [d for _, d in sub.degree()]
            if all(d == 2 for d in degs) and nx.is_connected(sub):
                count += 1
    return count

In [21]:
def graphlet_counts(g):
    '''considering graphlets of sze 3 and 4'''
    counts = {"triangle": 0, "path3": 0, "square": 0, "star4": 0, "clique4": 0}
    nodes = list(g.nodes())

    for trip in itertools.combinations(nodes, 3):
        sub =g.subgraph(trip)
        e =sub.number_of_edges()
        if e ==3 and nx.is_connected(sub):
            counts["triangle"] +=1
        elif e == 2 and nx.is_connected(sub):
            counts["path3"] +=1
    
    for quad in itertools.combinations(nodes, 4):
        sub =g.subgraph(quad)
        e =sub.number_of_edges()
        if e ==6:
            counts["clique4"] +=1
        elif e == 4:
            degs = [d for _,d in sub.degree()]
            if all(d == 2 for d in degs):
                counts["square"] +=1
        elif e ==3:
            degs =[d for _, d in sub.degree()]
            if sorted(degs) == [1,1,1,3]:
                counts["star4"] += 1
    
    return counts

In [22]:
def exact_isomorphism(g1,g2):
    GM = isomorphism.GraphMatcher(g1,g2,node_match=lambda x,y: x.get('label',None)==y.get('label',None))
    if GM.is_isomorphic():
        return True, dict(list(GM.mapping.items())[:8])
    else:
        return False, None

In [23]:
def analyze_group(dataset,group_indices,group_name="Group"):
    graphs =[]
    for idx in group_indices:
        data =dataset[idx]
        g = to_networkx(data,node_attrs=['x'])
        g =g.to_undirected()
        nx.set_node_attributes(g, {j:int(x[0]) for j,x in enumerate(data.x)},'label')
        graphs.append((idx,g))
    
    print(f"Analyzing {group_name} (size={len(graphs)})")
    for i in range(len(graphs)):
        for j in range(i+1,len(graphs)):
            id1,g1 =graphs[i]
            id2,g2 =graphs[j]
            deg_equal =degree_sequence(g1) ==degree_sequence(g2)
            twohop_equal =two_hop_neighbor_degrees(g1) ==two_hop_neighbor_degrees(g2)
            tri1, tri2 =triangle_count(g1),triangle_count(g2)
            four1, four2 =four_cycle_count(g1),four_cycle_count(g2)
            gc1, gc2 =graphlet_counts(g1),graphlet_counts(g2)
            iso, mapping =exact_isomorphism(g1,g2)
            if iso:
                print(f"Graph {id1} vs {id2}: Isomorphic(True))")
            else:
                reason = []
                if not deg_equal: 
                    reason.append("degree sequence")
                if not twohop_equal: 
                    reason.append("2-hop structure")
                if tri1 != tri2: 
                    reason.append(f"triangles={tri1} vs {tri2}")
                if four1 != four2: 
                    reason.append(f"4-cycles={four1} vs {four2}")
                if gc1 != gc2: 
                    reason.append(f"graphlet counts differ {gc1} vs {gc2}")
                reason = ", ".join(reason) if reason else "no obvious differnce but not isomorphic"
                print(f"Graph {id1} vs {id2}: WL claim is Fake ({reason})")


In [24]:
group1 = sorted_groups[0]
group2 = sorted_groups[1]
analyze_group(dataset, group1, "Group 1 (largest WL group)")
analyze_group(dataset, group2, "Group 2 (second largest WL group)")

Analyzing Group 1 (largest WL group) (size=8)
Graph 709 vs 1270: Isomorphic(True))
Graph 709 vs 1454: Isomorphic(True))
Graph 709 vs 1585: Isomorphic(True))
Graph 709 vs 1607: Isomorphic(True))
Graph 709 vs 1809: Isomorphic(True))
Graph 709 vs 1964: Isomorphic(True))
Graph 709 vs 1977: Isomorphic(True))
Graph 1270 vs 1454: Isomorphic(True))
Graph 1270 vs 1585: Isomorphic(True))
Graph 1270 vs 1607: Isomorphic(True))
Graph 1270 vs 1809: Isomorphic(True))
Graph 1270 vs 1964: Isomorphic(True))
Graph 1270 vs 1977: Isomorphic(True))
Graph 1454 vs 1585: Isomorphic(True))
Graph 1454 vs 1607: Isomorphic(True))
Graph 1454 vs 1809: Isomorphic(True))
Graph 1454 vs 1964: Isomorphic(True))
Graph 1454 vs 1977: Isomorphic(True))
Graph 1585 vs 1607: Isomorphic(True))
Graph 1585 vs 1809: Isomorphic(True))
Graph 1585 vs 1964: Isomorphic(True))
Graph 1585 vs 1977: Isomorphic(True))
Graph 1607 vs 1809: Isomorphic(True))
Graph 1607 vs 1964: Isomorphic(True))
Graph 1607 vs 1977: Isomorphic(True))
Graph 1809 

**Our results show that WL Test was accurate at identifying the isomophism in graphs in the groups we examined.However, in general WL test is necessary but insufficient condition for graph isomorphism i.e. if two graphs are isomorphic they must have same WL colors but converse is not guranteed. So there exists non isomorphic graphs that WL cannot distinguish (i.e. have same WL colors)**

## Proving the graphs in our dataset are not strongly Regular 

In [25]:
network_x_graphs=[]
for i,data in enumerate(dataset):
    g= to_networkx(data,node_attrs=['x'])
    g= g.to_undirected()
    nx.set_node_attributes(g,{j:int(x[0]) for j,x in enumerate(data.x)},'label')
    network_x_graphs.append(g)

In [26]:
srg_list=[]#list to store strongly regular graphs
for index, g in enumerate(network_x_graphs):
    try:
        result= distance_regular.is_strongly_regular(g)
        if result is not False:
            srg_list.append((index,result))
    except Exception as e:
        pass

In [28]:
print(f"Total graphs checked: {len(network_x_graphs)}")
print("Strongly regular graphs found:", srg_list)

Total graphs checked: 2000
Strongly regular graphs found: [(1482, True)]


In [ ]:
graph_index= 1482
data = dataset[graph_index]
G = to_networkx(data, to_undirected=True)
nx.set_node_attributes(G, {j:int(x[0]) for j,x in enumerate(data.x)},'label')
print(f"Graph {graph_index} has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

Graph 1482 has 5 nodes and 5 edges


In [30]:
original_hash = weisfeiler_lehman_hash(G)
edges= list(G.edges())
non_edges= list(nx.non_edges(G))
u,v = random.choice(edges)

In [31]:
G_removed= G.copy()
G_removed.remove_edge(u,v)
x,y =random.choice(non_edges)
G_added = G.copy()
G_added.add_edge(x,y)

In [32]:
GM_removed = nx.isomorphism.GraphMatcher(G,G_removed,node_match=lambda a,b: a['label']==b['label'])
GM_added = nx.isomorphism.GraphMatcher(G,G_added,node_match=lambda a,b: a['label']==b['label'])
print(f"Original vs Removed-edge graph: {GM_removed.is_isomorphic()}")
print(f"Original vs Added-edge graph  : {GM_added.is_isomorphic()}")

Original vs Removed-edge graph: False
Original vs Added-edge graph  : False
